In [ ]:
!pip -q install model-registry==0.2.15

In [ ]:
from model_registry import ModelRegistry
from model_registry.utils import S3Params
from model_registry.exceptions import StoreError
import os

In [ ]:
# Add your user name and cluster domain (apps.xxx) that are shared with you before

username = "fraud"
cluster_domain = "apps.ocpai.smrc.dsk"

In [ ]:
# Set up the model registry connection
model_registry_url = f"https://{username}-registry-rest.{cluster_domain}"
author_name = username

registry = ModelRegistry(server_address=model_registry_url, port=443, author=author_name, is_secure=False)

In [ ]:
# Model details we want to register
registered_model_name = "fraud"
version = "0.0.1"
model_path = "models/fraud/"

In [ ]:
s3_upload_params = S3Params(
    bucket_name=os.environ.get('AWS_S3_BUCKET'),
    s3_prefix="models/fraud/",
)

try:
    registered_model = registry.upload_artifact_and_register_model(
        name=registered_model_name,
        model_files_path=model_path,
        model_format_name="onnx",
        author=username,
        model_format_version="1",
        version=version,
        description="data",
        metadata={
            "accuracy": 0.3,
            "license": "apache-2.0"
        },
        upload_params=s3_upload_params
    )

    print(f"'{registered_model_name}' version '{version}' has been registered here: https://rhods-dashboard-redhat-ods-applications.{cluster_domain}/modelRegistry/{username}-registry/registeredModels/1/versions/{registry.get_model_version(registered_model_name, version).id}/details")

except StoreError:
    rmver = registry.get_model_version(registered_model_name, version)
    print(f"Model and version already exists:\n{rmver}")

In [ ]:
# Print the general info of registered model
model = registry.get_registered_model("fraud")
print("Registered Model:", model, "with ID", model.id)

In [ ]:
# Print the version info of registered model
version = registry.get_model_version("fraud", "0.0.1")
print("Model Version:", version, "with ID", version.id)

In [ ]:
# Print the artifact info of registered model
art = registry.get_model_artifact("fraud", "0.0.1")
print("Model Artifact:", art, "with ID", art.id)